# Notebooks pour Allgo
[Allgo](https://allgo.inria.fr/) est un service qui offre une interface web pour des outils en ligne de commande. Il dispense ainsi les utilisateurs d'installer les programmes en question et offre la possibilité de cacher leur code source et modèles (pour les outils ayant besoin d'un entraînement). Il permet donc : 
* D'évaluer des outils de recherche (par des groupes de recherche et des industriels ?)
* D'archiver un environnement d'exécution

Ce carnet est la suite du [premier](step_1.ipynb); ici, on se propose de partir de zéro et de faire toutes les requêtes à Allgo depuis notre carnet. 

## Speads

[Speads](https://allgo.inria.fr/app/speads) est un exemple d'application disponible sur Allgo. Speads segmente une conversation, identifie les locuteurs et tente de déterminer leur genre.

Par exemple, en partant de cette conversation :

In [ ]:
from IPython.display import Audio
Audio("conv1.mp3")

On obtient en sortie le tableau suivant :

In [ ]:
import csv
with open('conv1_speads.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        print(row)

Plutôt qu'un simple tableau, on voudrait obtenir un graphique de ce type:
![graphique](plot_1.png)

Ici par exemple, on voit bien plus facilement qu'on a cinq locuteurs dont deux sont des hommes.

Pour obtenir ce graphique, *et pour d'autres tâches d'analyse* -- il ne s'agit pas que de visualisation--, on a besoin d'un environnement plus puissant que l'UI d'Allgo, ce qui justifie le recours à un notebook type Jupyter. Par exemple, dans les données brutes, le genre du locuteur est mélangé avec son identifiant (comme dans `speaker4_F`), ce qui est problématique.

## Récupérer les données via l'API REST

Dans le [notebook précédent](step_1.ipynb), on était parti de données récupérées à la main sur Allgo.

Ici, on part simplement du fichier audio de conversation, et on veut interroger allgo directement (pour pouvoir facilement ajuster des paramètres, par exemple).

Allgo s'interroge comme suit:
* Le client `POST`e une requête (ID_application, paramètres, fichier(s) d'entrée)
* Réponse contenant un identifiant
* ... le client interroge le serveur pour suivre le job ...
* Réponses indiquant que le job est en cours, puis réponse contenant une liste de fichiers téléchargeables
* Le client télécharge le ou les fichiers qui l'intéressent


In [45]:
# La librairie requests est nécessaire (pip install requests)
import requests
TOKEN = "***REMOVED***" # privé, peut être regénéré
APP_ID = 99

def start_job(token, app_id, files, params=''):
    """Launches job, returns job ID"""
    headers = {'Authorization': 'Token token={}'.format(token)}
    data = {'job[webapp_id]': app_id}
    data['job[param]'] = params
    r = requests.post('https://allgo.inria.fr/api/v1/jobs', 
                      headers=headers,
                      files=files,
                      data=data)
    r.raise_for_status() # lever une exception en cas d'erreur 
    return r

def check_job(token, id):
    headers = {'Authorization': 'Token token={}'.format(token)}
    r = requests.get('https://allgo.inria.fr/api/v1/jobs/{}'.format(id),
                    headers=headers)
    r.raise_for_status()
    return r


L'API d'Allgo n'est pas super intuitive (?) -> 200 pour une requête qui finit en "error: no input file", notation étrange (`job[webapp_id]`, `files[0]`) etc.

In [46]:
files={'files[0]': open('conv1.mp3', 'rb')}
resp = start_job(token=TOKEN, app_id=APP_ID, files=files, params='-s 0.12')
#print(resp)
print(resp.text)

{"avg_time":30,"id":10212,"url":"https://allgo.inria.fr/api/v1/jobs/10212"}


In [49]:
# On vérifie la progression de la tâche
rcheck = check_job(token=TOKEN,id=resp.json()['id'])
print(rcheck)
print(rcheck.text)

<Response [200]>
{"10212":{"conv1_speads.txt":"https://allgo.inria.fr/datastore/248/99/9d51ce334fbf00e38a640b6d61185f429153f213/conv1_speads.txt","conv1.mp3":"https://allgo.inria.fr/datastore/248/99/9d51ce334fbf00e38a640b6d61185f429153f213/conv1.mp3","allgo.log":"https://allgo.inria.fr/datastore/248/99/9d51ce334fbf00e38a640b6d61185f429153f213/allgo.log"}}


In [ ]:
# On télécharge le fichier de résultat (et le log?)
with open('out/conv1_raw.tsv', 'w') as fb:
    pass


## Préparer les données

Ici, on corrige le problème évoqué plus haut (genre et identifiant du locuteur mélangés)

In [51]:
import csv
def preprocess_raw_speads_output(infile, outfile):
    with open(outfile, 'w') as dest:
        dest.write("ID\tGender\tStart\tEnd\n")
        with open(infile, 'r') as source:
            reader = csv.reader(source, delimiter='\t')
            for row in reader:
                dest.write('{}\t{}\t{}\t{}\n'.format(row[0][:-2], row[0][-1:], row[1], row[2]))

In [52]:
preprocess_raw_speads_output(infile='out/conv1_raw.tsv', outfile='out/conv1_processed.tsv')

Jetons un oeil aux données après traitement :

In [53]:
import pandas as pd
data = pd.read_csv('out/conv1_processed.tsv', sep='\t')
data


,ID,Gender,Start,End
0,speaker0,F,0.00000,1.09000
1,speaker0,F,1.65000,2.60000
2,speaker1,M,3.07000,4.56000
3,speaker1,M,7.35000,8.93000
4,speaker2,M,8.93000,9.30000
5,speaker3,F,9.30000,12.26000
6,speaker1,M,12.26000,15.21000
7,speaker4,M,15.21000,16.23000
8,speaker5,F,16.23000,20.74000
9,speaker6,F,20.74000,22.16000


## Visualiser la conversation avec Vega-Lite

In [54]:
import vega
vega.VegaLite({
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
      }
    }
  }
},
data)

... ce qui n'est pas le résultat attendu.

* Première surprise : pas de support stable de Vega-Lite dans JupyterLab (alors qu'il s'agit d'un plugin *core*). Au 3 octobre 2017, la bibliothèque python nécessaire n'est pas disponible par `pip` (et un build manuel échoue).
  - Donc, on ne peut embarquer la visualisation précédente que dans Jupyter Classic, à moins d'utiliser des appels bas niveau ? 
* Deuxième surprise : dans Jupyter Classic, le plugin Vega-Lite ne permet pas les visualisations superposées (*layered*). J'ai ouvert [un ticket](https://github.com/altair-viz/jupyter_vega/issues/38) à ce sujet.
* Apparemment, la version de vega-lite utilisée dans le plugin *core* est la v1 (la version actuelle est la v2).

On peut contourner ce problème avec une approche bas niveau (qu'on aimerait quand même éviter...)

In [55]:
def display_static_conversation(dataframe):
  from IPython.display import display
  import json
  bundle_contents = {
  "$schema": "https://vega.github.io/schema/vega-lite/v2.json",
  "description": "Conversation timeline",
  "data": {
    "values": json.loads(dataframe.to_json(orient='records'))
  },
  "mark": "bar",
  "encoding": {
    "y": {"field": "ID", "type": "nominal"},
    "x": {"field": "Start", "type": "quantitative"},
    "x2": {"field": "End", "type": "quantitative"},
    "color": {"field": "Gender", 
              "type": "nominal",
              "scale": {
                "domain": ["F","M"],
                 "range": ["#ff99ff","#4169e1"]
              }
          }
      }
  }
  mime_bundle = {'application/vnd.vegalite.v1+json': bundle_contents}
  display(mime_bundle, raw=True)

display_static_conversation(data)

(on note dans le type MIME que vegalite a une version de retard...)



## Discussion



## Annexe: nettoyage

In [ ]:
def clean():
    import os
    os.system('rm out/*')



In [ ]:
clean()